In [1]:
%pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
##IMPORTING ESSENTIALS##
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
###SEEING THE WEB

url = "https://www.iiserkol.ac.in/web/en/people/faculty/dms/#gsc.tab=0"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
html = response.text

print(html[:500])



<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="">
    <meta name="author" content="">

    <title>IISER Kolkata - dms</title>

    <!--<link href="//netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap.min.css" rel="stylesheet">-->
    
    <link href="/static/bootstrap/css/bootstrap.min.css" rel="styl


In [5]:
soup = BeautifulSoup(html, "lxml")

In [6]:
rows = soup.select("table.cms-plugin td")
print(f"Found {len(rows)} rows")

Found 18 rows


In [7]:
faculty_list = []
base_url = "https://www.iiserkol.ac.in/web/en/people/faculty/dms/#gsc.tab=0"

for td in soup.select("table.cms-plugin td"):
    name_tag = td.select_one("h4 a")
    if not name_tag:
        continue

    name = name_tag.get_text(strip=True)
    profile = name_tag["href"]
    if profile.startswith("/"):
        profile = "https://www.iiserkol.ac.in" + profile +"/#gsc.tab=0"

    # Get all text inside the td
    block_text = td.get_text(" ", strip=True)

    # Extract fields by string search
    department = ""
    research_area = ""
    email = ""

    if "Department:" in block_text:
        department = block_text.split("Department:")[1].split("Research Area:")[0].strip()
    if "Research Area:" in block_text:
        research_area = block_text.split("Research Area:")[1].split("Email:")[0].strip()
    if "Email:" in block_text:
        email = block_text.split("Email:")[1].strip()

    faculty_list.append({
                      "name": name,
                      "profile": profile,
                      "department": department,
                      "research_area": research_area,
                      "email": email
    })

print(faculty_list[:3])

for faculty in faculty_list:
    profile_url = faculty["profile"]
    if profile_url:
        if profile_url.startswith("/"):
            profile_url = base_url + profile_url

        try:
            resp = requests.get(profile_url, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")

            # Extract PhD info
            phd_text = ""
            for p in soup.select("ol li"):
                if "Ph.D." in p.get_text():
                    phd_text = p.get_text(strip=True)
            
            phd_text = ""
            for p in soup.select("ol li"):
                if "PhD" in p.get_text():
                    phd_text = p.get_text(strip=True)
            

            # Extract Postdoc info (optional, if present)
            postdoc_text = ""
            for p in soup.select("ol li"):
                if "Postdoc" in p.get_text():
                    postdoc_text = p.get_text(strip=True)

            faculty["phd"] = phd_text
            faculty["postdoc"] = postdoc_text

        except Exception as e:
            print(f"Error fetching {profile_url}: {e}")
            faculty["phd"] = ""
            faculty["postdoc"] = ""

    else:
        faculty["phd"] = ""
        faculty["postdoc"] = ""

[{'name': 'Anirban Banerjee', 'profile': 'https://www.iiserkol.ac.in/web/en/people/faculty/dms/anirban-banerjee/#gsc.tab=0', 'department': 'Mathematics and Statistics', 'research_area': 'Spectral graph and hypergraph theory, Structure and evolution of biological networks, Human brain functional networks', 'email': 'anirban.banerjee [AT] iiserkol.ac.in'}, {'name': 'Anirvan Chakraborty', 'profile': 'https://www.iiserkol.ac.in/web/en/people/faculty/dms/anirvan-c/#gsc.tab=0', 'department': 'Mathematics and Statistics', 'research_area': 'Statistics', 'email': 'anirvan.c [AT] iiserkol.ac.in'}, {'name': 'Arjun Paul', 'profile': 'https://www.iiserkol.ac.in/web/en/people/faculty/dms/arjun-paul/#gsc.tab=0', 'department': 'Mathematics and Statistics', 'research_area': 'Algebraic Geometry', 'email': 'arjun.paul [AT] iiserkol.ac.in'}]


In [8]:
df = pd.DataFrame(faculty_list)
df.to_csv("iiserkol_facultymath.csv", index=False, encoding="utf-8")

print(f"Saved {len(df)} faculty entries to iiserkol_faculty.csv")

Saved 18 faculty entries to iiserkol_faculty.csv
